In [1]:
import os
import datetime
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import random

import dataloader
import utils
from model import generate_model
from optimizer import Adam,SGD
from train_wrapper import train_epoch

d:\MyGit\CT_Classfication_Net\pymain\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = utils.load_config()

In [3]:
#seed 고정

seed = config['seed']
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
    

In [13]:
df_dataset = pd.read_csv(config['DATASET_PATH'])
df_dataset = df_dataset.dropna().reset_index(drop=True)

from sklearn.model_selection import train_test_split
X = df_dataset.drop(labels='label',axis=1)
Y = df_dataset['label']
np.random.seed(1004)
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,shuffle=True,stratify=None)

In [14]:
X_train

,filename,path
41,Dataset_normal_0000070_000000000000070_3.nii,Dataset/Normal/Dataset_normal_0000070_00000000...
475,Dataset_abnormal_0000286_000000000000286_3.nii,Dataset/Abnormal/Dataset_abnormal_0000286_0000...
159,Dataset_normal_0000252_000000000000252_3.nii,Dataset/Normal/Dataset_normal_0000252_00000000...
383,Dataset_abnormal_0000192_000000000000192_3.nii,Dataset/Abnormal/Dataset_abnormal_0000192_0000...
259,Dataset_abnormal_0000067_000000000000067_3.nii,Dataset/Abnormal/Dataset_abnormal_0000067_0000...
...,...,...
239,Dataset_abnormal_0000047_000000000000047_3.nii,Dataset/Abnormal/Dataset_abnormal_0000047_0000...
325,Dataset_abnormal_0000134_000000000000134_3.nii,Dataset/Abnormal/Dataset_abnormal_0000134_0000...
31,Dataset_normal_0000053_000000000000053_3.nii,Dataset/Normal/Dataset_normal_0000053_00000000...
11,Dataset_normal_0000019_000000000000019_3.nii,Dataset/Normal/Dataset_normal_0000019_00000000...


In [ ]:
batch_size = config['dataloader']['batch_size']
num_workers = config['dataloader']['num_workers']
learning_rate = config['train']['learning_rate']
epochs = config['train']['epochs']

model_name = config['model']['model_name']
model_depth = config['model']['model_depth']
config['save_datetime'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

In [ ]:
traindata=dataloader.CTDataset(X_train,y_train)
testdata=dataloader.CTDataset(X_test,y_test)

train_dataloader = DataLoader(traindata , batch_size=batch_size, shuffle=True, sampler = None, num_workers=num_workers,pin_memory = True)
test_dataloader = DataLoader(testdata , batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory = True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model, _ = generate_model(model_name=model_name,model_depth = model_depth,n_classes=2,resnet_shortcut='B',add_last_fc_num = 0)
model.to(device)

#패러럴 Train 미사용
#if torch.cuda.is_available():
#    model = nn.DataParallel(model, device_ids = [0,1,2,3])
#    model.to(device)
    
optimizer = Adam(model, learning_rate = learning_rate)
criterion_clf = nn.CrossEntropyLoss().to(device)

In [ ]:
utils.save_messgage(config
                    , model_name=model_name, model_depth = model_depth, n_classes=2, resnet_shortcut='B'
                    , optimizer = 'Adam', lr=learning_rate, criterion_clf='CrossEntropyLoss'
                    , message='비고')

train_epoch(device,train_dataloader,test_dataloader,test_dataloader,model,criterion_clf,optimizer,config,epoch = epochs,num_classes=2)